# Database Exploration and Analysis

This notebook provides comprehensive exploration of the Component Forge database, including schema analysis, data quality assessment, and relationship mapping.

## What You'll Explore

1. Database schema and table relationships
2. Data distribution and quality metrics
3. User activity patterns
4. Content analysis and categorization
5. Performance insights and optimization opportunities

## 1. Setup and Database Connection

In [ ]:
import sys
import os
import asyncio
from pathlib import Path
import json
from datetime import datetime, timedelta

# Add project paths
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
backend_src = project_root / 'backend' / 'src'
sys.path.insert(0, str(backend_src))

# Load environment
from dotenv import load_dotenv
load_dotenv(project_root / 'backend' / '.env')

# Data science imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Project imports
from core.database import database_health_check
from notebooks.utils.database_helpers import (
    sync_get_database_stats,
    sync_execute_query,
    sync_get_users_df,
    sync_get_documents_df,
    sync_get_conversations_df,
    sync_get_messages_df
)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("🔍 Database exploration environment ready!")

## 2. Database Health and Schema Overview

In [ ]:
# Check database health
health = await database_health_check()

print("🏥 Database Health Check:")
print(f"  Status: {health['status']}")
if health['status'] == 'healthy':
    print(f"  Database Version: {health.get('database_version', 'Unknown')}")
    print(f"  Tables: {health.get('table_count', 'Unknown')}")
    print(f"  Connection Time: {health.get('connection_time_ms', 'Unknown')}ms")
    print(f"  Pool Size: {health.get('pool_size', 'Unknown')}")
    print(f"  Active Connections: {health.get('checked_out_connections', 'Unknown')}")

# Get basic statistics
stats = sync_get_database_stats()
print(f"\n📊 Record Counts:")
for table, count in stats.items():
    print(f"  {table}: {count:,}")

# Visualize data distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of record counts
tables = list(stats.keys())
counts = list(stats.values())
colors = sns.color_palette("husl", len(tables))

bars = ax1.bar(tables, counts, color=colors)
ax1.set_title('Database Record Distribution')
ax1.set_ylabel('Number of Records')
ax1.tick_params(axis='x', rotation=45)

# Add value labels
for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(counts)*0.01,
             f'{count}', ha='center', va='bottom')

# Pie chart for proportions
if sum(counts) > 0:
    ax2.pie(counts, labels=tables, autopct='%1.1f%%', colors=colors)
    ax2.set_title('Data Distribution by Table')

plt.tight_layout()
plt.show()

## 3. Schema Analysis

In [ ]:
# Get table schema information
schema_query = """
SELECT 
    table_name,
    column_name,
    data_type,
    is_nullable,
    column_default
FROM information_schema.columns 
WHERE table_schema = 'public' 
AND table_name IN ('users', 'documents', 'conversations', 'messages', 'document_chunks', 'embedding_models', 'evaluation_runs')
ORDER BY table_name, ordinal_position
"""

schema_df = sync_execute_query(schema_query)

print("🗂️ Database Schema Overview:")
for table in schema_df['table_name'].unique():
    table_schema = schema_df[schema_df['table_name'] == table]
    print(f"\n📋 Table: {table}")
    print(f"  Columns: {len(table_schema)}")
    
    # Show key columns
    key_columns = table_schema[['column_name', 'data_type', 'is_nullable']].head()
    for _, col in key_columns.iterrows():
        nullable = "NULL" if col['is_nullable'] == 'YES' else "NOT NULL"
        print(f"    {col['column_name']}: {col['data_type']} ({nullable})")
    
    if len(table_schema) > 5:
        print(f"    ... and {len(table_schema) - 5} more columns")

# Analyze data types distribution
dtype_counts = schema_df['data_type'].value_counts()

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(dtype_counts.index[:10], dtype_counts.values[:10])  # Top 10 data types
ax.set_title('Most Common Data Types')
ax.set_ylabel('Number of Columns')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Detailed Table Analysis

### 4.1 Users Analysis

In [ ]:
# Load and analyze users
users_df = sync_get_users_df()

if len(users_df) > 0:
    print(f"👥 Users Analysis ({len(users_df)} users):")
    
    # Basic statistics
    active_users = users_df['is_active'].sum()
    admin_users = users_df['is_admin'].sum()
    total_logins = users_df['login_count'].sum()
    
    print(f"  Active users: {active_users}/{len(users_df)} ({active_users/len(users_df)*100:.1f}%)")
    print(f"  Admin users: {admin_users}/{len(users_df)} ({admin_users/len(users_df)*100:.1f}%)")
    print(f"  Total logins: {total_logins:,}")
    print(f"  Average logins per user: {total_logins/len(users_df):.1f}")
    
    # User activity distribution
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Login count distribution
    ax1.hist(users_df['login_count'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_title('Login Count Distribution')
    ax1.set_xlabel('Login Count')
    ax1.set_ylabel('Number of Users')
    
    # User types
    user_types = users_df.groupby(['is_admin', 'is_active']).size().reset_index(name='count')
    user_types['label'] = user_types.apply(
        lambda x: f"{'Admin' if x['is_admin'] else 'User'} ({'Active' if x['is_active'] else 'Inactive'})", 
        axis=1
    )
    ax2.pie(user_types['count'], labels=user_types['label'], autopct='%1.0f%%')
    ax2.set_title('User Type Distribution')
    
    # Registration timeline (if created_at is available)
    if 'created_at' in users_df.columns:
        users_df['created_date'] = pd.to_datetime(users_df['created_at']).dt.date
        daily_registrations = users_df.groupby('created_date').size()
        ax3.plot(daily_registrations.index, daily_registrations.values, marker='o')
        ax3.set_title('User Registration Timeline')
        ax3.set_xlabel('Date')
        ax3.set_ylabel('New Registrations')
        ax3.tick_params(axis='x', rotation=45)
    
    # Last login analysis (if available)
    if 'last_login_at' in users_df.columns and users_df['last_login_at'].notna().any():
        users_df['last_login_date'] = pd.to_datetime(users_df['last_login_at']).dt.date
        recent_logins = users_df['last_login_date'].value_counts().sort_index()
        ax4.bar(range(len(recent_logins)), recent_logins.values)
        ax4.set_title('Recent Login Activity')
        ax4.set_xlabel('Date')
        ax4.set_ylabel('Users Logged In')
    
    plt.tight_layout()
    plt.show()
    
    # Display user details
    print("\n👤 User Details:")
    display(users_df[['username', 'email', 'is_active', 'is_admin', 'login_count']].head())
else:
    print("No users found in database")

### 4.2 Documents Analysis

In [ ]:
# Load and analyze documents
docs_df = sync_get_documents_df()

if len(docs_df) > 0:
    print(f"📄 Documents Analysis ({len(docs_df)} documents):")
    
    # Basic statistics
    total_words = docs_df['word_count'].sum()
    avg_words = docs_df['word_count'].mean()
    total_size = docs_df['file_size'].sum()
    
    print(f"  Total words: {total_words:,}")
    print(f"  Average words per document: {avg_words:.0f}")
    print(f"  Total file size: {total_size/1024:.1f} KB")
    print(f"  Content types: {docs_df['content_type'].nunique()}")
    print(f"  Categories: {docs_df['category'].nunique()}")
    
    # Document metrics analysis
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Word count distribution
    ax1.hist(docs_df['word_count'], bins=15, alpha=0.7, color='lightgreen', edgecolor='black')
    ax1.axvline(avg_words, color='red', linestyle='--', label=f'Mean: {avg_words:.0f}')
    ax1.set_title('Document Length Distribution')
    ax1.set_xlabel('Word Count')
    ax1.set_ylabel('Number of Documents')
    ax1.legend()
    
    # Content types
    content_counts = docs_df['content_type'].value_counts()
    ax2.pie(content_counts.values, labels=content_counts.index, autopct='%1.0f%%')
    ax2.set_title('Content Type Distribution')
    
    # Categories
    category_counts = docs_df['category'].value_counts()
    ax3.bar(category_counts.index, category_counts.values)
    ax3.set_title('Document Categories')
    ax3.set_xlabel('Category')
    ax3.set_ylabel('Number of Documents')
    ax3.tick_params(axis='x', rotation=45)
    
    # File size vs word count correlation
    docs_df['file_size_kb'] = docs_df['file_size'] / 1024
    ax4.scatter(docs_df['word_count'], docs_df['file_size_kb'], alpha=0.7)
    ax4.set_title('Word Count vs File Size')
    ax4.set_xlabel('Word Count')
    ax4.set_ylabel('File Size (KB)')
    
    # Add correlation coefficient
    correlation = docs_df['word_count'].corr(docs_df['file_size_kb'])
    ax4.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=ax4.transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Processing status analysis
    status_counts = docs_df['processing_status'].value_counts()
    print(f"\n⚙️ Processing Status:")
    for status, count in status_counts.items():
        print(f"  {status}: {count} documents ({count/len(docs_df)*100:.1f}%)")
    
    # Category analysis
    print(f"\n📂 Category Analysis:")
    category_stats = docs_df.groupby('category').agg({
        'word_count': ['count', 'sum', 'mean'],
        'file_size': 'sum'
    }).round(2)
    
    display(category_stats)
else:
    print("No documents found in database")

### 4.3 Conversations and Messages Analysis

In [ ]:
# Load and analyze conversations
conversations_df = sync_get_conversations_df()
messages_df = sync_get_messages_df()

if len(conversations_df) > 0:
    print(f"💬 Conversations Analysis ({len(conversations_df)} conversations):")
    
    # Basic statistics
    total_messages = messages_df.shape[0] if len(messages_df) > 0 else 0
    total_tokens = conversations_df['total_tokens_used'].sum()
    avg_messages = conversations_df['message_count'].mean()
    unique_users = conversations_df['user_id'].nunique()
    
    print(f"  Total messages: {total_messages:,}")
    print(f"  Total tokens used: {total_tokens:,}")
    print(f"  Average messages per conversation: {avg_messages:.1f}")
    print(f"  Active users: {unique_users}")
    
    # Conversation analysis
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Message count distribution
    ax1.hist(conversations_df['message_count'], bins=10, alpha=0.7, color='orange', edgecolor='black')
    ax1.axvline(avg_messages, color='red', linestyle='--', label=f'Mean: {avg_messages:.1f}')
    ax1.set_title('Messages per Conversation')
    ax1.set_xlabel('Message Count')
    ax1.set_ylabel('Number of Conversations')
    ax1.legend()
    
    # Token usage distribution
    ax2.hist(conversations_df['total_tokens_used'], bins=10, alpha=0.7, color='purple', edgecolor='black')
    ax2.set_title('Token Usage Distribution')
    ax2.set_xlabel('Total Tokens Used')
    ax2.set_ylabel('Number of Conversations')
    
    # Model usage
    model_counts = conversations_df['model_name'].value_counts()
    ax3.pie(model_counts.values, labels=model_counts.index, autopct='%1.0f%%')
    ax3.set_title('Model Usage Distribution')
    
    # Conversations by user
    user_conv_counts = conversations_df['username'].value_counts()
    ax4.bar(user_conv_counts.index, user_conv_counts.values)
    ax4.set_title('Conversations by User')
    ax4.set_xlabel('Username')
    ax4.set_ylabel('Number of Conversations')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Message role analysis
    if len(messages_df) > 0:
        print(f"\n🗨️ Message Analysis:")
        role_counts = messages_df['role'].value_counts()
        for role, count in role_counts.items():
            print(f"  {role}: {count:,} messages ({count/len(messages_df)*100:.1f}%)")
        
        # Token usage by role (if available)
        if 'total_tokens' in messages_df.columns:
            token_by_role = messages_df.groupby('role')['total_tokens'].agg(['sum', 'mean', 'count']).fillna(0)
            print(f"\n🎯 Token Usage by Role:")
            display(token_by_role)
    
    # Temperature analysis
    print(f"\n🌡️ Temperature Settings:")
    temp_stats = conversations_df['temperature'].describe()
    for stat, value in temp_stats.items():
        print(f"  {stat}: {value:.3f}")
else:
    print("No conversations found in database")

## 5. Cross-Table Relationship Analysis

In [ ]:
# Analyze relationships between tables
relationship_query = """
SELECT 
    u.username,
    u.is_admin,
    u.login_count,
    COUNT(DISTINCT c.id) as conversation_count,
    COUNT(DISTINCT d.id) as document_count,
    COUNT(m.id) as message_count,
    SUM(CASE WHEN m.role = 'user' THEN 1 ELSE 0 END) as user_messages,
    SUM(CASE WHEN m.role = 'assistant' THEN 1 ELSE 0 END) as assistant_messages,
    SUM(m.total_tokens) as total_tokens,
    AVG(m.total_tokens) as avg_tokens_per_message,
    MAX(c.last_message_at) as last_activity
FROM users u
LEFT JOIN conversations c ON u.id = c.user_id
LEFT JOIN documents d ON u.id = d.uploaded_by_id
LEFT JOIN messages m ON c.id = m.conversation_id
GROUP BY u.id, u.username, u.is_admin, u.login_count
ORDER BY conversation_count DESC, total_tokens DESC
"""

user_activity_df = sync_execute_query(relationship_query)

if len(user_activity_df) > 0:
    print("🔗 User Activity Correlation Analysis:")
    
    # Display user activity summary
    display(user_activity_df[[
        'username', 'conversation_count', 'message_count', 
        'document_count', 'total_tokens', 'last_activity'
    ]].head())
    
    # Correlation analysis
    numeric_cols = ['login_count', 'conversation_count', 'message_count', 'total_tokens']
    correlation_matrix = user_activity_df[numeric_cols].corr()
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Correlation heatmap
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=ax1)
    ax1.set_title('User Activity Correlations')
    
    # Login count vs conversations
    ax2.scatter(user_activity_df['login_count'], user_activity_df['conversation_count'], alpha=0.7)
    ax2.set_xlabel('Login Count')
    ax2.set_ylabel('Conversation Count')
    ax2.set_title('Logins vs Conversations')
    
    # Messages vs tokens
    valid_data = user_activity_df.dropna(subset=['message_count', 'total_tokens'])
    if len(valid_data) > 0:
        ax3.scatter(valid_data['message_count'], valid_data['total_tokens'], alpha=0.7)
        ax3.set_xlabel('Message Count')
        ax3.set_ylabel('Total Tokens')
        ax3.set_title('Messages vs Token Usage')
    
    # User type analysis
    admin_stats = user_activity_df.groupby('is_admin')[numeric_cols].mean()
    admin_stats.plot(kind='bar', ax=ax4)
    ax4.set_title('Activity by User Type')
    ax4.set_xlabel('Is Admin')
    ax4.set_ylabel('Average Count')
    ax4.tick_params(axis='x', rotation=0)
    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical insights
    print(f"\n📊 Key Insights:")
    
    # Most active users
    most_active = user_activity_df.nlargest(3, 'conversation_count')
    print(f"  Most active users (by conversations):")
    for _, user in most_active.iterrows():
        print(f"    {user['username']}: {user['conversation_count']} conversations, {user['message_count']} messages")
    
    # Token usage leaders
    if user_activity_df['total_tokens'].notna().any():
        token_leaders = user_activity_df.nlargest(3, 'total_tokens')
        print(f"  Top token users:")
        for _, user in token_leaders.iterrows():
            tokens = user['total_tokens'] or 0
            print(f"    {user['username']}: {tokens:,.0f} tokens")
    
    # Activity patterns
    active_users = user_activity_df[user_activity_df['conversation_count'] > 0]
    if len(active_users) > 0:
        avg_conversations = active_users['conversation_count'].mean()
        avg_messages = active_users['message_count'].mean()
        print(f"  Average conversations per active user: {avg_conversations:.1f}")
        print(f"  Average messages per active user: {avg_messages:.1f}")
else:
    print("No user activity data available")

## 6. Data Quality Assessment

In [ ]:
# Data quality checks
print("🔍 Data Quality Assessment:")

quality_issues = []

# Check users table
if len(users_df) > 0:
    null_emails = users_df['email'].isnull().sum()
    duplicate_emails = users_df['email'].duplicated().sum()
    null_usernames = users_df['username'].isnull().sum()
    duplicate_usernames = users_df['username'].duplicated().sum()
    
    print(f"\n👥 Users Quality:")
    print(f"  Null emails: {null_emails}")
    print(f"  Duplicate emails: {duplicate_emails}")
    print(f"  Null usernames: {null_usernames}")
    print(f"  Duplicate usernames: {duplicate_usernames}")
    
    if null_emails > 0: quality_issues.append(f"Users: {null_emails} null emails")
    if duplicate_emails > 0: quality_issues.append(f"Users: {duplicate_emails} duplicate emails")
    if null_usernames > 0: quality_issues.append(f"Users: {null_usernames} null usernames")
    if duplicate_usernames > 0: quality_issues.append(f"Users: {duplicate_usernames} duplicate usernames")

# Check documents table
if len(docs_df) > 0:
    null_titles = docs_df['title'].isnull().sum()
    zero_word_count = (docs_df['word_count'] == 0).sum()
    negative_file_size = (docs_df['file_size'] < 0).sum()
    failed_processing = (docs_df['processing_status'] == 'failed').sum()
    
    print(f"\n📄 Documents Quality:")
    print(f"  Null titles: {null_titles}")
    print(f"  Zero word count: {zero_word_count}")
    print(f"  Negative file size: {negative_file_size}")
    print(f"  Failed processing: {failed_processing}")
    
    if null_titles > 0: quality_issues.append(f"Documents: {null_titles} null titles")
    if zero_word_count > 0: quality_issues.append(f"Documents: {zero_word_count} zero word count")
    if negative_file_size > 0: quality_issues.append(f"Documents: {negative_file_size} negative file sizes")
    if failed_processing > 0: quality_issues.append(f"Documents: {failed_processing} failed processing")

# Check conversations table
if len(conversations_df) > 0:
    null_titles = conversations_df['title'].isnull().sum()
    zero_messages = (conversations_df['message_count'] == 0).sum()
    negative_tokens = (conversations_df['total_tokens_used'] < 0).sum()
    
    print(f"\n💬 Conversations Quality:")
    print(f"  Null titles: {null_titles}")
    print(f"  Zero message count: {zero_messages}")
    print(f"  Negative token count: {negative_tokens}")
    
    if null_titles > 0: quality_issues.append(f"Conversations: {null_titles} null titles")
    if zero_messages > 0: quality_issues.append(f"Conversations: {zero_messages} zero message count")
    if negative_tokens > 0: quality_issues.append(f"Conversations: {negative_tokens} negative tokens")

# Check messages table
if len(messages_df) > 0:
    null_content = messages_df['content_length'].isnull().sum() if 'content_length' in messages_df.columns else 0
    invalid_roles = (~messages_df['role'].isin(['user', 'assistant', 'system'])).sum()
    
    print(f"\n🗨️ Messages Quality:")
    print(f"  Invalid roles: {invalid_roles}")
    
    if invalid_roles > 0: quality_issues.append(f"Messages: {invalid_roles} invalid roles")

# Summary
print(f"\n📋 Quality Summary:")
if quality_issues:
    print(f"  ⚠️ Issues found: {len(quality_issues)}")
    for issue in quality_issues:
        print(f"    - {issue}")
else:
    print(f"  ✅ No major quality issues detected")

# Data completeness analysis
completeness_data = []

for df_name, df in [('users', users_df), ('documents', docs_df), ('conversations', conversations_df)]:
    if len(df) > 0:
        completeness = {}
        for col in df.columns:
            if col in df.select_dtypes(include=[np.number]).columns:
                non_null_pct = (df[col].notna().sum() / len(df)) * 100
                completeness[col] = non_null_pct
        
        if completeness:
            completeness_data.append({
                'table': df_name,
                'avg_completeness': np.mean(list(completeness.values())),
                'min_completeness': min(completeness.values()),
                'columns_checked': len(completeness)
            })

if completeness_data:
    completeness_df = pd.DataFrame(completeness_data)
    print(f"\n📊 Data Completeness (Numeric Columns):")
    display(completeness_df)

    # Visualize completeness
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(completeness_df['table'], completeness_df['avg_completeness'])
    ax.set_title('Data Completeness by Table')
    ax.set_ylabel('Average Completeness (%)')
    ax.set_ylim(0, 100)
    
    # Add value labels
    for bar, value in zip(bars, completeness_df['avg_completeness']):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{value:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

## 7. Performance Insights

In [ ]:
# Performance analysis
print("⚡ Performance Insights:")

# Table size analysis
table_size_query = """
SELECT 
    schemaname,
    tablename,
    attname as column_name,
    n_distinct,
    correlation
FROM pg_stats 
WHERE schemaname = 'public' 
AND tablename IN ('users', 'documents', 'conversations', 'messages')
ORDER BY tablename, attname
"""

try:
    stats_df = sync_execute_query(table_size_query)
    
    if len(stats_df) > 0:
        print(f"\n📈 Column Statistics:")
        
        # Show high-cardinality columns (potential indexing candidates)
        high_cardinality = stats_df[stats_df['n_distinct'] > 100].sort_values('n_distinct', ascending=False)
        if len(high_cardinality) > 0:
            print(f"  High-cardinality columns (good for indexing):")
            for _, row in high_cardinality.head().iterrows():
                print(f"    {row['tablename']}.{row['column_name']}: {row['n_distinct']} distinct values")
        
        # Show columns with high correlation (potential clustering candidates)
        high_correlation = stats_df[abs(stats_df['correlation']) > 0.1].sort_values('correlation', ascending=False)
        if len(high_correlation) > 0:
            print(f"  Columns with high correlation (clustering candidates):")
            for _, row in high_correlation.head().iterrows():
                print(f"    {row['tablename']}.{row['column_name']}: {row['correlation']:.3f} correlation")
except Exception as e:
    print(f"  Could not retrieve column statistics: {e}")

# Index analysis
index_query = """
SELECT 
    schemaname,
    tablename,
    indexname,
    indexdef
FROM pg_indexes 
WHERE schemaname = 'public' 
AND tablename IN ('users', 'documents', 'conversations', 'messages')
ORDER BY tablename, indexname
"""

try:
    indexes_df = sync_execute_query(index_query)
    
    if len(indexes_df) > 0:
        print(f"\n🗂️ Current Indexes:")
        for table in indexes_df['tablename'].unique():
            table_indexes = indexes_df[indexes_df['tablename'] == table]
            print(f"  {table}: {len(table_indexes)} indexes")
            for _, idx in table_indexes.iterrows():
                # Extract index type and columns from definition
                idx_def = idx['indexdef']
                if 'UNIQUE' in idx_def:
                    idx_type = "UNIQUE"
                elif 'btree' in idx_def:
                    idx_type = "BTREE"
                else:
                    idx_type = "OTHER"
                print(f"    {idx['indexname']}: {idx_type}")
except Exception as e:
    print(f"  Could not retrieve index information: {e}")

# Query performance recommendations
print(f"\n💡 Performance Recommendations:")

# Based on data patterns
if len(conversations_df) > 0:
    avg_messages = conversations_df['message_count'].mean()
    if avg_messages > 10:
        print(f"  - Consider partitioning messages table (avg {avg_messages:.1f} messages per conversation)")

if len(docs_df) > 0:
    avg_word_count = docs_df['word_count'].mean()
    if avg_word_count > 1000:
        print(f"  - Consider document chunking for large documents (avg {avg_word_count:.0f} words)")

# General recommendations based on data volume
total_records = sum(stats.values())
if total_records > 10000:
    print(f"  - Consider connection pooling for {total_records:,} total records")
    print(f"  - Monitor query performance with pg_stat_statements")
    print(f"  - Consider read replicas for heavy read workloads")

if len(messages_df) > 100:
    print(f"  - Consider archiving old messages ({len(messages_df)} total messages)")
    print(f"  - Implement message pagination for large conversations")

print(f"\n🎯 Optimization Priorities:")
print(f"  1. Index frequently queried columns (user_id, conversation_id)")
print(f"  2. Monitor and optimize slow queries")
print(f"  3. Implement appropriate caching strategies")
print(f"  4. Consider data archiving for old records")
print(f"  5. Set up monitoring and alerting for database performance")

## 8. Summary and Action Items

Based on this database exploration, here are the key findings and recommended actions:

### 📊 Data Overview
- Database health and connection status
- Record distribution across tables
- Data quality assessment results

### 🔍 Key Insights
- User activity patterns and engagement levels
- Document content analysis and categorization
- Conversation and token usage patterns
- Cross-table relationships and correlations

### ⚠️ Quality Issues
- Review any data quality issues identified
- Implement data validation rules
- Set up data quality monitoring

### ⚡ Performance Opportunities
- Index optimization based on query patterns
- Data archiving strategies for growth
- Monitoring and alerting setup

### 🚀 Next Steps
1. **Performance Analysis**: Run `notebooks/evaluation/01_performance_analysis.ipynb`
2. **User Behavior**: Explore `notebooks/exploration/02_user_behavior.ipynb`
3. **Query Optimization**: Identify and optimize slow queries
4. **Monitoring Setup**: Implement database monitoring and alerting

This exploration provides a solid foundation for understanding your data and optimizing the Component Forge platform! 🎯